<a href="https://colab.research.google.com/github/RiverGumSecurity/AILabs/blob/main/022_LargeLanguageModels/SFT_infotech.ipynb" target="_new"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Information Technology Dataset Supervised Fine Tuning of Llama3

* Google colab does not have unsloth installed by default
* Hugging Face API Key/Token will need to exist in keystore for colab or local disk otherwise
* We set a max_seq_length parameter to set the token vector size.  Padding will occur if less than this size
* When text streaming in Google Colab, wrapping does not work so we are forced to inject some CSS using an iPython callback


In [11]:
import torch
import warnings
import os
import sys
import pathlib
try:
    import unsloth
except:
    if 'google.colab' in sys.modules:
        !pip install unsloth

# suppresses some noisy warnings which are just annoying
warnings.filterwarnings('ignore')
max_seq_length = 4096

# Setup Hugging Face Credentials.
HF_APIKEY = ''
if 'google.colab' in sys.modules:
    from google.colab import userdata
    HF_APIKEY = userdata.get('HF_APIKEY')
else:
    with open(pathlib.Path.home() / '.hfkey') as hf:
        HF_APIKEY = hf.read().strip()
if not HF_APIKEY:
    print('[-] ERROR: Cannot continue without Hugging Face API Key')
    sys.exit(0)
os.environ['HF_TOKEN'] = HF_APIKEY

model, tokenizer = unsloth.FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = None, load_in_4bit = True
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
# This code fixes a word wrapping issue
# in Google Co-Lab on Text Streamer output.
from warnings import warn
from IPython import get_ipython
from IPython.display import HTML, display
from transformers import TextStreamer

def enable_word_wrap(*args, **kwargs):
    display(HTML('''
    <style>
        pre {
            white-space: pre-wrap;
        }
    </style>
    '''))

def register_ipython_callback_once(event_name, cb):
    ev = get_ipython().events
    cb_unregs = [cb_old for cb_old in ev.callbacks[event_name] if cb_old.__name__ == cb.__name__]
    if len(cb_unregs) == 1 and cb.__code__ == cb_unregs[0].__code__:
        return
    for cb_old in cb_unregs:
        warn(f'Removing unexpected callback {cb_old}.')
        ev.unregister(event_name, cb_old)
    ev.register(event_name, cb)

register_ipython_callback_once('pre_run_cell', enable_word_wrap)

In [4]:
# PEFT MODEL
model = unsloth.FastLanguageModel.get_peft_model(
    model,
    r = 16,               ## 8 or 16 is reasonable
    lora_alpha = 32,      ## 32 on a GPU with min 16GB Ram
    lora_dropout = 0.07,  ## between 0.05 and 1.0

    ## best to leave below params alone
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"],
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.07.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def format_prompts(p):
    # these are provided as lists
    instructions = p['prompt']
    outputs      = p['messages']
    texts = []
    for ins, outp in zip(instructions, outputs):
        text = prompt.format(ins, '', outp[1]['content']) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts }

from datasets import load_dataset
ds = load_dataset("gussieIsASuccessfulWarlock/information_technology_instruct_mcq_2481", split = "train")
ds = ds.map(format_prompts, batched = True,)
ds = ds.remove_columns(["messages"])

instruct.jsonl:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/784 [00:00<?, ? examples/s]

Map:   0%|          | 0/784 [00:00<?, ? examples/s]

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

newmodel = model
trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = ds,
    args = TrainingArguments(
        ## hyper-params we usually can tune
        learning_rate = 2e-4,             ## between 1e-4 and 5e-4
        per_device_train_batch_size = 4,  ## 2, 4, or 8
        gradient_accumulation_steps = 8,  ## 4 up to 8
        num_train_epochs = 1,             ## max_steps overrides this
        max_steps = 10,                   ## single Epoch / 50 steps or more
        weight_decay = 0.01,              ## 0.01 is conservative and good for not overfitting

        ## params below are typically not modified
        warmup_steps = 5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/784 [00:00<?, ? examples/s]

In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 784 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.685200
2,1.591000
3,1.722400
4,1.669000
5,1.497100
6,1.313200
7,1.224700
8,1.196500
9,1.125900
10,1.045500


In [8]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory/max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)

print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
7.412 GB of memory reserved.
82.7171 seconds used for training.
1.38 minutes used for training.
Peak reserved memory = 7.412 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 18.738 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [12]:
model.push_to_hub('l3-infotech')
tokenizer.push_to_hub('l3-infotech')

README.md:   0%|          | 0.00/605 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Saved model to https://huggingface.co/l3-infotech


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [13]:
unsloth.FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        "What is an IP address and subnet mask?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

<|begin_of_text|>
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
What is an IP address and subnet mask?

### Input:


### Response:
An IP address is a series of numbers separated by dots, and a subnet mask is a series of numbers separated by dots that is used to determine the network address of a device.
<|end_of_text|>
